In [248]:
import pandas as pd
import urllib
import requests
import json
import sys
import io
import shapely
import base64
import urllib.request
import functools
import geopandas as gp
import fiona
import numpy as np
import shapely
from fiona.crs import from_epsg
from functools import reduce
import itertools

%matplotlib inline

## start here if you want to replicate downloading and transforming the data, otherwise skip to the bottom

In [249]:
myAPI = <<your api key>>

In [281]:
variables = ['S2503_C01_001E','S2503_C01_002E','S2503_C01_003E','S2503_C01_004E','S2503_C01_005E','S2503_C01_006E',
             'S2503_C01_007E' , 'S2503_C01_008E' ,'S2503_C01_009E','S2503_C01_010E','S2503_C01_011E',
             'S2503_C01_012E','S2503_C01_013E']

In [292]:
#read in in the variables available. the info you need is in the 1year ACS data
url = "https://api.census.gov/data/2016/acs/acs5/subject/variables.json"
resp = requests.request('GET', url)
aff1y = json.loads(resp.text)

affkeys = np.array(list(aff1y['variables'].keys()))

#extracting variables of B28002 that contain "broadband"
[(k, aff1y['variables'][k]['label'])  for k in affkeys if k.startswith ("S2503_C01_") ]

[('S2503_C01_014E',
  'Occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!Less than $300'),
 ('S2503_C01_015E',
  'Occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!$300 to $499'),
 ('S2503_C01_012E',
  'Occupied housing units!!Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!$150,000 or more'),
 ('S2503_C01_013E',
  'Occupied housing units!!Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars)'),
 ('S2503_C01_018E',
  'Occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!$1,000 to $1,499'),
 ('S2503_C01_019E',
  'Occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!$1,500 to $1,999'),
 ('S2503_C01_016E',
  'Occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!$500 to $799'),
 ('S2503_C01_017E',
  'Occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!$800 to $999'),
 ('S2503_C01_010E',
  'Occupied housing units!!Estimate!!HOUSEHOLD INCOME IN THE PAST 12 

In [250]:
#subject group url:
#https://api.census.gov/data/2017/acs/acs1/subject/groups/S2503.html

key1 = 'S2503_C01_001E,S2503_C01_002E,S2503_C01_003E,S2503_C01_004E,S2503_C01_005E,S2503_C01_006E,S2503_C01_007E'
key2= ',S2503_C01_008E,S2503_C01_009E,S2503_C01_010E,S2503_C01_011E,S2503_C01_012E,S2503_C01_013E'
year = '2017'

counties = [ 5, 47, 61, 81, 85]

url = "https://api.census.gov/data/2017/acs/acs5/subject?get=NAME," \
    + key1 + key2 + "&for=tract:*&in=state:36&in=county:*&key=" + myAPI
resp = requests.request('GET', url).content
income = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))

income = income[income['county'].isin(counties)]


income.head(1)

,NAME,S2503_C01_001E,S2503_C01_002E,S2503_C01_003E,S2503_C01_004E,S2503_C01_005E,S2503_C01_006E,S2503_C01_007E,S2503_C01_008E,S2503_C01_009E,S2503_C01_010E,S2503_C01_011E,S2503_C01_012E,S2503_C01_013E,state,county,tract,Unnamed: 17
21,"Census Tract 170.09, Richmond County, New York",1588,0,24,67,62,65,42,135,247,87,379,480,104393,36,85,17009,NaN


In [251]:
income.rename(columns={'S2503_C01_001E' : 'total_count', 'S2503_C01_002E':'bracket_1',
                       'S2503_C01_003E':'bracket_2','S2503_C01_004E':'bracket_3',
                       'S2503_C01_005E':'bracket_4','S2503_C01_006E':'bracket_5'
                       ,'S2503_C01_007E':'bracket_6','S2503_C01_008E':'bracket_7',
                       'S2503_C01_009E':'bracket_8','S2503_C01_010E': 'bracket_9', 
                       'S2503_C01_011E':'bracket10','S2503_C01_012E':'bracket_11',
                       'S2503_C01_013E':'median_household_income'}, inplace=True)

In [252]:
income.head(1)

,NAME,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,bracket_9,bracket10,bracket_11,median_household_income,state,county,tract,Unnamed: 17
21,"Census Tract 170.09, Richmond County, New York",1588,0,24,67,62,65,42,135,247,87,379,480,104393,36,85,17009,NaN


In [253]:
path = '/Users/katharinevoorhees/CAPSTONE/taxizone_censustract_key.csv'

key = pd.read_csv(path)

key.head()

,index_right,OBJECTID,zone,tractid,tractnum
0,1968,2.0,Jamaica Bay,36081107202,107202
1,222,3.0,Allerton/Pelham Gardens,36005031000,31000
2,224,3.0,Allerton/Pelham Gardens,36005031400,31400
3,226,3.0,Allerton/Pelham Gardens,36005031800,31800
4,225,3.0,Allerton/Pelham Gardens,36005031600,31600


In [254]:
income_taxi = income.merge(key, left_on='tract', right_on='tractnum')

In [255]:
income_taxi.head()

,NAME,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,...,median_household_income,state,county,tract,Unnamed: 17,index_right,OBJECTID,zone,tractid,tractnum
0,"Census Tract 170.09, Richmond County, New York",1588,0,24,67,62,65,42,135,247,...,104393,36,85,17009,NaN,2121,5.0,Arden Heights,36085017009,17009
1,"Census Tract 207, Richmond County, New York",1787,379,122,142,85,36,177,148,234,...,31250,36,85,20700,NaN,2136,187.0,Port Richmond,36085020700,20700
2,"Census Tract 207, Richmond County, New York",1787,379,122,142,85,36,177,148,234,...,31250,36,85,20700,NaN,501,189.0,Prospect Heights,36047020700,20700
3,"Census Tract 207, Kings County, New York",2179,82,25,16,48,84,168,170,130,...,113574,36,47,20700,NaN,2136,187.0,Port Richmond,36085020700,20700
4,"Census Tract 207, Kings County, New York",2179,82,25,16,48,84,168,170,130,...,113574,36,47,20700,NaN,501,189.0,Prospect Heights,36047020700,20700


In [256]:
zone_income = income_taxi.groupby('OBJECTID').sum().reset_index()

In [257]:
zone_income.head()

,OBJECTID,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,...,bracket10,bracket_11,median_household_income,state,county,tract,Unnamed: 17,index_right,tractid,tractnum
0,2.0,0,0,0,0,0,0,0,0,0,...,0,0,-666666666,36,81,107202,0.0,1968,36081107202,107202
1,3.0,18922,791,1136,2045,890,1308,1867,2117,2960,...,2336,1621,920583,576,290,532000,0.0,3762,576080532000,532000
2,4.0,19981,1403,1677,2318,1300,1277,1649,2068,2787,...,2391,1751,-666017378,468,671,31005,0.0,14589,468793031005,31005
3,5.0,9048,200,161,303,216,219,517,1016,1563,...,1809,2081,345970,144,340,68034,0.0,8482,144340068034,68034
4,6.0,25623,1293,836,1183,1051,982,1753,2252,3066,...,3874,6297,-1332166657,612,1029,84402,0.0,35250,613445084402,84402


In [258]:
zone_income = zone_income.iloc[:, 0:13]

zone_income.tail()

,OBJECTID,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,bracket_9,bracket10,bracket_11
256,259.0,29056,1080,1193,2128,1394,1267,2321,3964,4969,3814,3908,3018
257,260.0,80409,3352,4583,5980,4443,3714,7928,10290,13055,9472,9871,7721
258,261.0,3487,71,28,88,32,39,41,152,367,256,903,1510
259,262.0,23650,733,721,705,697,600,1002,2034,2911,2570,4182,7495
260,263.0,33964,656,603,1042,890,1175,1451,2006,5049,4255,6637,10200


In [259]:
col = [ 'bracket_1', 'bracket_2', 'bracket_3','bracket_4', 'bracket_5', 'bracket_6', 'bracket_7', 'bracket_8',
       'bracket_9', 'bracket10', 'bracket_11']
for i in col: 
    zone_income[i + '_prob'] = zone_income[i]/zone_income['total_count']

In [260]:
zone_income.set_index('OBJECTID', inplace=True)

In [261]:
zone_income.head()

,total_count,bracket_1,bracket_2,bracket_3,bracket_4,bracket_5,bracket_6,bracket_7,bracket_8,bracket_9,...,bracket_2_prob,bracket_3_prob,bracket_4_prob,bracket_5_prob,bracket_6_prob,bracket_7_prob,bracket_8_prob,bracket_9_prob,bracket10_prob,bracket_11_prob
OBJECTID,,,,,,,,,,,,,,,,,,,,,
2.0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,18922,791,1136,2045,890,1308,1867,2117,2960,1851,...,0.060036,0.108075,0.047035,0.069126,0.098668,0.111880,0.156432,0.097823,0.123454,0.085667
4.0,19981,1403,1677,2318,1300,1277,1649,2068,2787,1360,...,0.083930,0.116010,0.065062,0.063911,0.082528,0.103498,0.139483,0.068065,0.119664,0.087633
5.0,9048,200,161,303,216,219,517,1016,1563,963,...,0.017794,0.033488,0.023873,0.024204,0.057140,0.112290,0.172745,0.106432,0.199934,0.229996
6.0,25623,1293,836,1183,1051,982,1753,2252,3066,3036,...,0.032627,0.046169,0.041018,0.038325,0.068415,0.087890,0.119658,0.118487,0.151192,0.245756


In [262]:
zone_income = zone_income.iloc[:, 12:]

zone_income.tail()

,bracket_1_prob,bracket_2_prob,bracket_3_prob,bracket_4_prob,bracket_5_prob,bracket_6_prob,bracket_7_prob,bracket_8_prob,bracket_9_prob,bracket10_prob,bracket_11_prob
OBJECTID,,,,,,,,,,,
259.0,0.037170,0.041059,0.073238,0.047976,0.043605,0.079880,0.136426,0.171015,0.131264,0.134499,0.103868
260.0,0.041687,0.056996,0.074370,0.055255,0.046189,0.098596,0.127971,0.162357,0.117798,0.122760,0.096022
261.0,0.020361,0.008030,0.025237,0.009177,0.011184,0.011758,0.043590,0.105248,0.073416,0.258962,0.433037
262.0,0.030994,0.030486,0.029810,0.029471,0.025370,0.042368,0.086004,0.123087,0.108668,0.176829,0.316913
263.0,0.019315,0.017754,0.030680,0.026204,0.034595,0.042722,0.059063,0.148657,0.125280,0.195413,0.300318


## start here if you already have table csv

In [295]:
zone_income.to_csv('zone_income_probabilities.csv')

In [299]:
df = pd.read_csv('zone_income_probabilities.csv')
df.set_index('OBJECTID', inplace=True)

In [305]:
#define wage brackets

bucket = [1,2,3,4,5,6,7,8,9,10,11]
bracket = {1:(0,5000), 2:(5000,9999), 3:(10000,14999), 4:(15000,19999), 5:(20000,24999), 6:(25000,34999), 7:(35000,49999),
           8:(50000,74999), 9:(75000,99999), 10:(100000,149999), 11:(150000,300000)}

#define function

def wage(df_wage,row):
    """returns randomly chosen income,
    weighted by income bracket probability in that taxi zone"""
    prob_list = []
    for i in df_wage.iloc[row, 0:]:
        prob_list.append(i)
    choice = np.random.choice(bucket, p=prob_list)
    return np.random.randint(bracket[choice][0], size=None)
    
#test     

wage(df, 4)   

17183